In [ ]:
import os
from pymongo import MongoClient
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()

In [ ]:

client = MongoClient(os.getenv('MONGODB_URI'))
db_name = os.getenv('MONGODB_DB')
collection_name = os.getenv('MONGODB_COLL')
collection = client[db_name][collection_name]
print(db_name,collection_name)
prompt = "Which movies is scary?"

print('True')


In [ ]:

# initialize vector store
vectorstore = MongoDBAtlasVectorSearch(
    collection=collection, 
    embedding=OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY')), 
    index_name="vector_index", 
    embedding_key='plot_embedding',
    text_key='plot'
)

# perform a similarity search between a query and the ingested documents
docs = vectorstore.similarity_search(prompt,5)
print(prompt)
for doc in docs:
    print(doc.metadata['title'],'-', doc.page_content)

docs_with_score = vectorstore.similarity_search_with_score(query=prompt,k=2)
print(" ")
print(prompt)

for doc, score in docs_with_score:
    metadata = doc.metadata  
    # print(metadata)
    print(metadata['title'], '-', doc.page_content)
    print(score)

client.close()
